---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [20]:
# env setup
import pandas as pd
import numpy as np
# from beakerx import *
# preprocessing
from sklearn import preprocessing
# from imblearn.under_sampling import RandomUnderSampler #can't be installed to coursera python kernel
# model selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
# evaluation metrics
from sklearn.metrics import precision_recall_curve, roc_curve, auc, classification_report
# visuals
# import matplotlib.pyplot as plt

def blight_model():
    
    # read data
    data_train = (pd.read_csv('train.csv',
                             encoding = 'ISO-8859-1',
                             dtype={'zip_code': str,
                                    'non_us_str_code': str,
                                    'grafitti_status': str,
                                    'violator_name':str,
                                    'mailing_address_str_number': str}
                            )
                  .set_index('ticket_id')
                 )

    data_test = (pd.read_csv('test.csv',
                            dtype={'zip_code': str,
                                    'non_us_str_code': str,
                                    'grafitti_status': str,
                                    'violator_name':str,
                                    'mailing_address_str_number': str}
                            )
                 .set_index('ticket_id')
                )

    data_address = pd.read_csv('addresses.csv').set_index('ticket_id')
    data_latlon = pd.read_csv('latlons.csv')

    # merge address and GPS coordinates
    data_address_latlon = pd.merge(data_address, data_latlon, how = 'left', left_on = 'address', right_on = 'address')

    # missing GPS coordinates
#     data_address_latlon[data_address_latlon['lat'].isnull()]
#     data_address_latlon['lat'].isnull().sum()

    # add GPS info to training dataset
    data_train = pd.merge(data_train, data_address_latlon, how = 'left', left_index = True, right_index = True)
    data_test = pd.merge(data_test, data_address_latlon, how = 'left', left_index = True, right_index = True)

    # missing GPS coordinates
#     data_train[data_train['lat'].isnull()]
#     print('Missing rows with GPS coordinates in data_train:', data_train['lat'].isnull().sum())
#     print('Missing rows with GPS coordinates in data_test:', data_test['lat'].isnull().sum())

    # Missing rows with GPS coordinates in data_train: 17
    # Missing rows with GPS coordinates in data_test: 44978
    # seems to be useless

    # stats
#     print('===================================================')
#     print('Number of rows in data_train: ', len(data_train))
#     print('Number of rows in data_test: ', len(data_test))
#     print('Number of rows in data_address_latlon: ', len(data_address_latlon))
#     print('===================================================')
#     print('Distribution of target variable in data_train: ')
#     print(data_train['compliance'].value_counts())
#     print('===================================================')
#     print('Number of missing values in target variable: ', data_train['compliance'].isnull().sum())

    # Distribution of target variable in data_train: 
    # 0.0    148283
    # 1.0     11597
    # Name: compliance, dtype: int64
    # imbalanced data set

    # data preprocessing
    # subset training data to hold only flag 0 and 1 for target variable
    data_train = data_train[data_train['compliance'].notnull()]

    # convert ticket_issued_date to DateTime and map its development over time
    data_train['ticket_issued_date'] = pd.to_datetime(data_train['ticket_issued_date'])
    data_train['ticket_issued_date_adj'] = (
        (data_train['ticket_issued_date'] - data_train['ticket_issued_date'].min())
        /
        (data_train['ticket_issued_date'].max() - data_train['ticket_issued_date'].min())
    )

    data_test['ticket_issued_date'] = pd.to_datetime(data_test['ticket_issued_date'])
    data_test['ticket_issued_date_adj'] = (
        (data_test['ticket_issued_date'] - data_test['ticket_issued_date'].min())
        /
        (data_test['ticket_issued_date'].max() - data_test['ticket_issued_date'].min())
    )


    # define predictors and target value
    X_blight = data_train[['disposition',
                           'ticket_issued_date_adj',
                           'fine_amount',                   
                           'admin_fee',
                           'state_fee',
                           'late_fee',
                           'discount_amount',
                           'clean_up_cost',
                           'judgment_amount']]

    X_blight_test = data_test[['disposition',
                           'ticket_issued_date_adj',
                           'fine_amount',                   
                           'admin_fee',
                           'state_fee',
                           'late_fee',
                           'discount_amount',
                           'clean_up_cost',
                           'judgment_amount']]

    y_blight = data_train[['compliance']]
    y_blight_reshaped = y_blight.values.reshape(-1)

    # use LabelEncoder() to transform categorical variables to numerical ones
    le = preprocessing.LabelEncoder()
    X_blight_encoded = X_blight[['disposition']].apply(le.fit_transform)
    X_blight_encoded_test = X_blight_test[['disposition']].apply(le.fit_transform)

    X_blight_encoded = pd.merge(X_blight[['ticket_issued_date_adj',
                                          'fine_amount',                   
                                           'admin_fee',
                                           'state_fee',
                                           'late_fee',
                                           'discount_amount',
                                           'clean_up_cost',
                                           'judgment_amount']],
                                X_blight_encoded,
                                how = 'left',
                                left_index = True,
                                right_index = True)

    X_blight_encoded_test = pd.merge(X_blight_test[['ticket_issued_date_adj',
                                          'fine_amount',                   
                                           'admin_fee',
                                           'state_fee',
                                           'late_fee',
                                           'discount_amount',
                                           'clean_up_cost',
                                           'judgment_amount']],
                                X_blight_encoded_test,
                                how = 'left',
                                left_index = True,
                                right_index = True)

    # undersampling (imbalanced data set)
#     under_sampling = RandomUnderSampler(random_state=0)
#     X_resampled, y_resampled = under_sampling.fit_resample(X_blight_encoded, y_blight_reshaped)

    # train / test split for data_train dataset
#     X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=0) 
    X_train, X_test, y_train, y_test = train_test_split(X_blight_encoded, y_blight_reshaped, random_state=0)
    
    # train the classifier
    logreg = LogisticRegression(C = 100, solver = 'lbfgs', random_state=0).fit(X_train, y_train)

    # predict the target value on unseen data
    logreg_prediction = logreg.predict(X_test)

    # calculate the probabilities
    logreg_probabilities = logreg.predict_proba(X_test)[:,1] # for the positive category, blight ticket will be paid on time (compliance = 1)

    # cross-validation 5-fold
    # use default metric = accuracy as scoring metric
#     print('Mean score >>> Cross-validation (accuracy)', cross_val_score(logreg, X_resampled, y_resampled, cv=20).mean())
#     # use precision as scoring metric
#     print('Mean score >>> Cross-validation (precision)', cross_val_score(logreg, X_resampled, y_resampled, cv=20, scoring = 'precision').mean())
#     # use recall as scoring metric
#     print('Mean score >>> Cross-validation (recall)', cross_val_score(logreg, X_resampled, y_resampled, cv=20, scoring = 'recall').mean())
#     # use AUC as scoring metric
#     print('Mean score >>> Cross-validation (AUC)', cross_val_score(logreg, X_resampled, y_resampled, cv=20, scoring = 'roc_auc').mean())
    # AUC
    fpr_logreg, tpr_logreg, _ = roc_curve(y_test, logreg_probabilities)
    roc_auc_logreg = auc(fpr_logreg, tpr_logreg)
#     print('AUC for Logistic regresssion: ', roc_auc_logreg)
#     AUC for Logistic regresssion:  0.7572014240697578

    # classification report
#     print(classification_report(y_test, logreg_prediction, target_names=['not 1', '1']))

    # predict the values of target variable using data_test
    logreg_prediction_test = logreg.predict(X_blight_encoded_test)

    # calculate the probabilities
    logreg_probabilities_test = logreg.predict_proba(X_blight_encoded_test)[:,1]

    # result
    logreg_result_test = pd.Series(logreg_probabilities_test, index = X_blight_encoded_test.index)
#     print('Probability that the blight ticket will be paid on time using Logistic Regression: ')
#     print(logreg_result_test.head())
#     print()

#     # stats
#     unique, counts = np.unique(logreg_prediction_test, return_counts=True)
#     print('Predictions on TEST data using Logistic Regression: ', dict(zip(unique, counts)))
    
    return logreg_result_test

In [22]:
# blight_model()

In [13]:
# # Random Forests
# # train the classifier
# rndm_forrest = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0).fit(X_train, y_train)

# # predict the target value on unseen data
# rndm_forrest_prediction = rndm_forrest.predict(X_test)

# # calculate the probabilities
# rndm_forrest_probabilities = rndm_forrest.predict_proba(X_test)[:,1] # for the positive category, blight ticket will be paid on time (compliance = 1)

# # cross-validation 5-fold
# # use default metric = accuracy as scoring metric
# print('Mean score >>> Cross-validation (accuracy)', cross_val_score(rndm_forrest, X_resampled, y_resampled, cv=20).mean())
# # use precision as scoring metric
# print('Mean score >>> Cross-validation (precision)', cross_val_score(rndm_forrest, X_resampled, y_resampled, cv=20, scoring = 'precision').mean())
# # use recall as scoring metric
# print('Mean score >>> Cross-validation (recall)', cross_val_score(rndm_forrest, X_resampled, y_resampled, cv=20, scoring = 'recall').mean())
# # use AUC as scoring metric
# print('Mean score >>> Cross-validation (AUC)', cross_val_score(rndm_forrest, X_resampled, y_resampled, cv=20, scoring = 'roc_auc').mean())

# # AUC
# fpr_rndm_forrest, tpr_rndm_forrest, _ = roc_curve(y_test, rndm_forrest_probabilities)
# roc_auc_rndm_forrest = auc(fpr_rndm_forrest, tpr_rndm_forrest)
# print('AUC for Random Forrest Classifier: ', roc_auc_rndm_forrest)

# # classification report
# print(classification_report(y_test, rndm_forrest_prediction, target_names=['not 1', '1']))

# # predict the values of target variable using data_test
# rndm_forrest_prediction_test = rndm_forrest.predict(X_blight_encoded_test)

# # calculate the probabilities
# rndm_forrest_probabilities_test = rndm_forrest.predict_proba(X_blight_encoded_test)[:,1]

# # result
# rndm_forrest_result_test = pd.Series(rndm_forrest_probabilities_test, index = X_blight_encoded_test.index)
# print('Probability that the blight ticket will be paid on time using Random Forrest Classifier: ')
# print(rndm_forrest_result_test.head())
# print()

# # stats
# unique, counts = np.unique(rndm_forrest_prediction_test, return_counts=True)
# print('Predictions on TEST data using Random Forrest Classifier: ', dict(zip(unique, counts)))

In [14]:
# # Gradient Boosting
# # train the classifier
# gbm = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)

# # predict the target value on unseen data
# gbm_prediction = gbm.predict(X_test)

# # calculate the probabilities
# gbm_probabilities = gbm.predict_proba(X_test)[:,1] # for the positive category, blight ticket will be paid on time (compliance = 1)

# # cross-validation 5-fold
# # use default metric = accuracy as scoring metric
# print('Mean score >>> Cross-validation (accuracy)', cross_val_score(gbm, X_resampled, y_resampled, cv=20).mean())
# # use precision as scoring metric
# print('Mean score >>> Cross-validation (precision)', cross_val_score(gbm, X_resampled, y_resampled, cv=20, scoring = 'precision').mean())
# # use recall as scoring metric
# print('Mean score >>> Cross-validation (recall)', cross_val_score(gbm, X_resampled, y_resampled, cv=20, scoring = 'recall').mean())
# # use AUC as scoring metric
# print('Mean score >>> Cross-validation (AUC)', cross_val_score(gbm, X_resampled, y_resampled, cv=20, scoring = 'roc_auc').mean())

# # AUC
# fpr_gbm, tpr_gbm, _ = roc_curve(y_test, gbm_probabilities)
# roc_auc_gbm = auc(fpr_gbm, tpr_gbm)
# print('AUC for Gradient Boosting Classifier: ', roc_auc_gbm)

# # classification report
# print(classification_report(y_test, gbm_prediction, target_names=['not 1', '1']))

# # predict the values of target variable using data_test
# gbm_prediction_test = gbm.predict(X_blight_encoded_test)

# # calculate the probabilities
# gbm_probabilities_test = gbm.predict_proba(X_blight_encoded_test)[:,1]

# # result
# gbm_result_test = pd.Series(gbm_probabilities_test, index = X_blight_encoded_test.index)
# print('Probability that the blight ticket will be paid on time using Gradient Boosting Classifier: ')
# print(gbm_result_test.head())
# print()

# # stats
# unique, counts = np.unique(gbm_prediction_test, return_counts=True)
# print('Predictions on TEST data using Gradient Boosting Classifier: ', dict(zip(unique, counts)))

In [15]:
# Support Vector Machines
# # train the classifier
# svc = SVC(gamma = 'scale', random_state=0, probability = True).fit(X_train, y_train)

# # predict the target value on unseen data
# svc_prediction = svc.predict(X_test)

# # calculate the probabilities
# svc_probabilities = svc.predict_proba(X_test)[:,1] # for the positive category, blight ticket will be paid on time (compliance = 1)

# # cross-validation 5-fold
# # use default metric = accuracy as scoring metric
# print('Mean score >>> Cross-validation (accuracy)', cross_val_score(svc, X_resampled, y_resampled, cv=20).mean())
# # use AUC as scoring metric
# print('Mean score >>> Cross-validation (AUC)', cross_val_score(svc, X_resampled, y_resampled, cv=20, scoring = 'roc_auc').mean())
# # use recall as scoring metric
# print('Mean score >>> Cross-validation (recall)', cross_val_score(svc, X_resampled, y_resampled, cv=20, scoring = 'recall').mean())

# # AUC
# fpr_svc, tpr_svc, _ = roc_curve(y_test, svc_probabilities)
# roc_auc_svc = auc(fpr_svc, tpr_svc)
# print('AUC for Support Vector Classifier: ', roc_auc_svc)

# # classification report
# print(classification_report(y_test, svc_prediction, target_names=['not 1', '1']))

# it takes too long to run with probability set to True

In [19]:
# # given test script to test the outcome by size, type etc.
# res = 'Data type Test: '
# res += ['Failed: type(bm) should Series\n','Passed\n'][type(logreg_result_test)==pd.Series]
# res += 'Data shape Test: '
# res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(logreg_result_test)==61001]
# res += 'Data Values Test: '
# res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((logreg_result_test<=1.) & (logreg_result_test>=0.))]
# res += 'Data Values type Test: '
# res += ['Failed: bm.dtype should be float\n','Passed\n'][str(logreg_result_test.dtype).count('float')>0]
# res += 'Index type Test: '
# res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(logreg_result_test.index)==pd.Int64Index]
# res += 'Index values type Test: '
# res += ['Failed: type(bm.index[0]) should be int64\n','Passed\n'][str(type(logreg_result_test.index[0])).count("int64")>0]

# res += 'Output index shape test:'
# res += ['Failed, bm.index.shape should be (61001,)\n','Passed\n'][logreg_result_test.index.shape==(61001,)]

# res += 'Output index test: '
# if logreg_result_test.index.shape==(61001,):
#     res +=['Failed\n','Passed\n'][all(pd.read_csv('readonly/test.csv',usecols=[0],index_col=0).sort_index().index.values==logreg_result_test.sort_index().index.values)]
# else:
#     res+='Failed'
# print(res)

Data type Test: Passed
Data shape Test: Passed
Data Values Test: Passed
Data Values type Test: Passed
Index type Test: Passed
Index values type Test: Passed
Output index shape test:Passed
Output index test: Passed

